**Import required libraries and scripts**

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *
import numpy as np
import os

software = '/home/tony/DockM8/software'
protein_file = '/home/tony/DockM8/wocondock_main/2o1x_A_apo_protoss.pdb'
ref_file = '/home/tony/DockM8/wocondock_main/2o1x_A_lig_protoss.sdf'
docking_library = '/home/tony/DockM8/wocondock_main/Selection_of_FCHGroup_LeadLike.sdf'
docking_programs = ['GNINA', 'SMINA', 'PLANTS']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
rescoring_functions = ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'SCORCH', 'RTMScore']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
ncpus = int(os.cpu_count()/2)
#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

[12:04:13] Initializing Normalizer


The working directory has been set to: /home/tony/igf1r
The folder: /home/tony/igf1r/temp already exists


In [ ]:
pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')

In [ ]:
pocket = GetPocket(ref_file, protein_file, 8)

In [ ]:
cleaned_pkasolver_df = prepare_library(docking_library, id_column, software, 'pkasolver', ncpus)

In [ ]:
all_poses = docking(protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses)

In [ ]:
docking_splitted(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses, ncpus)

In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

In [ ]:
for metric in clustering_metrics:
    cluster_futures(f'{metric}', 'KMedoids', w_dir, protein_file, all_poses, ncpus)

**Rescoring**

The file containing all the cluster centers is then rescored using all scoring functions available (GNINA, Vina, AutoDock4, PLP, CHEMPLP, RF-Score-VS). The rescored output is return as a dataframe.

In [2]:
for metric in clustering_metrics:
    rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring_functions, 1, ncpus)

The folder: /home/tony/igf1r/temp/rescoring_RMSD_clustered already exists

[2023-Mar-21 22:44:58]: /gnina_rescoring folder already exists, skipping gnina rescoring

[2023-Mar-21 22:44:58]: /AD4_rescoring folder already exists, skipping AD4 rescoring

[2023-Mar-21 22:44:58]: /chemplp_rescoring folder already exists, skipping chemplp rescoring

[2023-Mar-21 22:44:58]: /rfscorevs_rescoring folder already exists, skipping rfscorevs rescoring

[2023-Mar-21 22:44:58]: /LinF9_rescoring folder already exists, skipping LinF9 rescoring
The folder: /home/tony/igf1r/temp/rescoring_RMSD_clustered/SCORCH_rescoring/ was created

[2023-Mar-21 22:44:58]: Converting protein file to .pdbqt ...

[2023-Mar-21 22:45:08]: Converting SDF file RMSD_clustered.sdf to .pdbqt files...
Converted 25409 molecules.

[2023-Mar-21 22:46:02]: Rescoring with SCORCH


**Final ranking methods**

This code calculates the final ranking of compounds using various methods.
*Method 1* : Calculates ECR value for each cluster center, then outputs the top ranked center.
*Method 2* : Calculates ECR value for each cluster center, then outputs the average ECR value for each compound.
*Method 3* : Calculates the average rank of each compound, then ouputs the corresponding ECR value for each compound.
*Method 6* : Calculates Z-score for each cluster center, then ouputs the top ranked center.
*Method 7* : Calculates Z-score for each cluster center, then ouputs the average Z-score for each compound.

All methods are then combined into a single dataframe for comparison purposes.

In [ ]:
apply_consensus_methods(w_dir, clustering_metrics)

In [ ]:
calculate_EFs(w_dir, docking_library)